In [1]:
!pip install pandas
!pip install sklearn

Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas
import numpy as np
import warnings
warnings.filterwarnings('ignore')
data = pandas.read_csv('StudentsPerformance.csv', index_col='gender')
data_sel = data.loc[:, data.columns.isin(['gender', 'race/ethnicity', 'parental level of education', 
                                          'lunch', 'test preparation course', 'math score', 'reading score','writing score'])]
data_sel['test preparation course'] = np.where(data_sel['test preparation course'] == 'none', 0, 1)

data_sel['lunch'] = np.where(data_sel['lunch'] == 'free/reduced', 0,1)

data_sel['race/ethnicity'] = np.where(data_sel['race/ethnicity'] == 'group A',  0, data_sel['race/ethnicity'])
data_sel['race/ethnicity'] = np.where(data_sel['race/ethnicity'] == 'group B',  1, data_sel['race/ethnicity'])
data_sel['race/ethnicity'] = np.where(data_sel['race/ethnicity'] == 'group C',  2, data_sel['race/ethnicity'])
data_sel['race/ethnicity'] = np.where(data_sel['race/ethnicity'] == 'group D',  3, data_sel['race/ethnicity'])
data_sel['race/ethnicity'] = np.where(data_sel['race/ethnicity'] == 'group E',  4, data_sel['race/ethnicity'])

data_sel['parental level of education'] = np.where(data_sel['parental level of education'] == 'some college',  0, data_sel['parental level of education'])
data_sel['parental level of education'] = np.where(data_sel['parental level of education'] == 'some high school',  1, data_sel['parental level of education'])
data_sel['parental level of education'] = np.where(data_sel['parental level of education'] == 'high school',  2, data_sel['parental level of education'])
data_sel['parental level of education'] = np.where(data_sel['parental level of education'] == "bachelor's degree",  3, data_sel['parental level of education'])
data_sel['parental level of education'] = np.where(data_sel['parental level of education'] == "associate's degree",  4, data_sel['parental level of education'])
data_sel['parental level of education'] = np.where(data_sel['parental level of education'] == "master's degree",  5, data_sel['parental level of education'])

data_sel = data_sel.dropna()
data_sel['writing score'] = np.where(data_sel['writing score'] > np.average(data_sel['writing score']) , 0, 1)
writing_score = data_sel.loc[:, data_sel.columns.isin(['writing score'])]
X = data_sel.loc[:, data_sel.columns.isin(['gender', 'race/ethnicity', 'parental level of education', 'lunch', 
                                           'test preparation course', 'math score', 'reading score'])]
X

,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score
gender,,,,,,
female,1,3,1,0,72,72
female,2,0,1,1,69,90
female,1,5,1,0,90,95
male,0,4,0,0,47,57
male,2,0,1,0,76,78
...,...,...,...,...,...,...
female,4,5,1,1,88,99
male,2,2,0,0,62,55
female,2,2,0,1,59,71


In [5]:
#опорные векторы
from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(X, writing_score, test_size=.33, random_state=5)
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

svm = SVC()
parameters = {'kernel':('linear', 'rbf'), 'C':(0.25,0.5,0.75,1),'gamma': (1,2,3,'auto'),'decision_function_shape':('ovo','ovr'),'shrinking':(True,False)}
clf = GridSearchCV(svm, parameters)
clf.fit(x_train,y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': (0.25, 0.5, 0.75, 1),
                         'decision_function_shape': ('ovo', 'ovr'),
                         'gamma': (1, 2, 3, 'auto'),
                         'kernel': ('linear', 'rbf'),
                         'shrinking': (True, False)})

In [6]:
print("f1:"+str(np.average(cross_val_score(clf, x_validation, y_validation, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(clf, x_validation, y_validation, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(clf, x_validation, y_validation, scoring='recall'))))

f1:0.9144850613243941


precision:0.920411613960001


recall:0.9142857142857143


In [7]:
#случайный лес
from sklearn.ensemble import RandomForestClassifier 
param_grid = { 'n_estimators': [50,100,150],'max_features': ['auto'],'max_depth' : list(range(1, 10)), 'criterion' :['gini']}
RFC = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv= 5, refit = True) 
RFC.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'max_features': ['auto'],
                         'n_estimators': [50, 100, 150]})

In [10]:
print("f1:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation,scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='recall'))))

f1:0.913311163073254


precision:0.9454301075268818


recall:0.9085714285714286
